In [1]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    port="5433",
    database="krankenhaus",
    user="postgres",
    password="postgres"
)

cur = conn.cursor()


In [2]:
drop_tables = """
-- =========================
-- DROP TABLES
-- =========================

DROP TABLE IF EXISTS Event_Medikament CASCADE;
DROP TABLE IF EXISTS Arzt_Event CASCADE;
DROP TABLE IF EXISTS Event CASCADE;
DROP TABLE IF EXISTS Behandlung CASCADE;
DROP TABLE IF EXISTS Medikament CASCADE;
DROP TABLE IF EXISTS Bett CASCADE;
DROP TABLE IF EXISTS Reserve CASCADE;
DROP TABLE IF EXISTS Pflegekraft CASCADE;
DROP TABLE IF EXISTS Arzt CASCADE;
DROP TABLE IF EXISTS Patient CASCADE;
DROP TABLE IF EXISTS Raum CASCADE;
DROP TABLE IF EXISTS Station CASCADE;
DROP TABLE IF EXISTS Krankenhaus CASCADE;
"""



In [3]:
conn.rollback()

cur.execute(drop_tables)
conn.commit()



In [4]:
create_tables = """
CREATE TABLE Krankenhaus (
    KH_ID INT PRIMARY KEY,
    Strasse VARCHAR(100) NOT NULL,
    Hausnummer VARCHAR(10) NOT NULL,
    Postleitzahl VARCHAR(10) NOT NULL,
    Stadt VARCHAR(50) NOT NULL,
    Name VARCHAR(100) NOT NULL,
    CHECK (Postleitzahl ~ '^[0-9]{4}$')
);

CREATE TABLE Station (
    Station_ID INT PRIMARY KEY,
    Name VARCHAR(50) NOT NULL,
    Kapazitaet INT NOT NULL,
    KH_ID INT NOT NULL,
    CHECK (Kapazitaet > 0),
    FOREIGN KEY (KH_ID) REFERENCES Krankenhaus(KH_ID)
);

CREATE TABLE Raum (
    Raumnummer INT PRIMARY KEY,
    Station_ID INT NOT NULL,
    Art VARCHAR(50),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Pflegekraft (
    Pflege_ID INT PRIMARY KEY,
    Telefonnummer VARCHAR(30),
    Vorname VARCHAR(50) NOT NULL,
    Nachname VARCHAR(50) NOT NULL,
    Station_ID INT NOT NULL,
    CHECK (Telefonnummer IS NULL OR Telefonnummer ~ '^[0-9+ ]+$'),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Patient (
    Patient_ID INT PRIMARY KEY,
    Vorname VARCHAR(50) NOT NULL,
    Nachname VARCHAR(50) NOT NULL,
    Versicherungsnummer VARCHAR(30) UNIQUE NOT NULL,
    Geburtsdatum DATE NOT NULL,
    Strasse VARCHAR(100),
    Hausnummer VARCHAR(10),
    Postleitzahl VARCHAR(10),
    Stadt VARCHAR(50),
    Elektronisches_Patientendossier_ID VARCHAR(50) UNIQUE,
    Station_ID INT,
    CHECK (Postleitzahl IS NULL OR Postleitzahl ~ '^[0-9]{4}$'),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Bett (
    Bett_ID INT PRIMARY KEY,
    Status VARCHAR(20) NOT NULL,
    Patient_ID INT,
    Station_ID INT NOT NULL,
    CHECK (Status IN ('frei', 'belegt')),
    FOREIGN KEY (Patient_ID) REFERENCES Patient(Patient_ID),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Arzt (
    Arzt_ID INT PRIMARY KEY,
    Telefonnummer VARCHAR(30),
    Vorname VARCHAR(50) NOT NULL,
    Nachname VARCHAR(50) NOT NULL,
    ist_extern BOOLEAN NOT NULL,
    Station_ID INT,
    CHECK (Telefonnummer IS NULL OR Telefonnummer ~ '^[0-9+ ]+$'),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Reserve (
    Reserve_ID INT PRIMARY KEY,
    Telefonnummer VARCHAR(30),
    Arzt_ID INT,
    Station_ID INT,
    Pflegekraft_ID INT,
    CHECK (Telefonnummer IS NULL OR Telefonnummer ~ '^[0-9+ ]+$'),
    FOREIGN KEY (Arzt_ID) REFERENCES Arzt(Arzt_ID),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID),
    FOREIGN KEY (Pflegekraft_ID) REFERENCES Pflegekraft(Pflege_ID)
);

CREATE TABLE Behandlung (
    Behandlungs_ID INT PRIMARY KEY,
    abgeschlossen BOOLEAN NOT NULL,
    Beschreibung VARCHAR(255),
    Start_Datum DATE NOT NULL,
    End_Datum DATE,
    Patient_ID INT NOT NULL,
    FOREIGN KEY (Patient_ID) REFERENCES Patient(Patient_ID),
    CHECK (End_Datum IS NULL OR End_Datum >= Start_Datum)
);

CREATE TABLE Medikament (
    Medikament_ID INT PRIMARY KEY,
    Wirkstoff VARCHAR(100),
    Name VARCHAR(100) NOT NULL   
);

CREATE TABLE Event (
    Event_ID INT PRIMARY KEY,
    Uhrzeit TIME NOT NULL,
    Datum DATE NOT NULL,
    RaumID INT NOT NULL,
    Behandlung_ID INT NOT NULL,
    FOREIGN KEY (RaumID) REFERENCES Raum(Raumnummer),
    FOREIGN KEY (Behandlung_ID) REFERENCES Behandlung(Behandlungs_ID)
);

CREATE TABLE Arzt_Event (
    Arzt_ID INT NOT NULL,
    Event_ID INT NOT NULL,
    PRIMARY KEY (Arzt_ID, Event_ID),
    FOREIGN KEY (Arzt_ID) REFERENCES Arzt(Arzt_ID),
    FOREIGN KEY (Event_ID) REFERENCES Event(Event_ID)
);

CREATE TABLE Event_Medikament (
    Event_ID INT NOT NULL,
    Medikament_ID INT NOT NULL,
    PRIMARY KEY (Event_ID, Medikament_ID),
    FOREIGN KEY (Event_ID) REFERENCES Event(Event_ID),
    FOREIGN KEY (Medikament_ID) REFERENCES Medikament(Medikament_ID)
);
"""
cur.execute(create_tables)
conn.commit()


In [5]:
import random
import datetime
from faker import Faker

fake = Faker("de_CH")
random.seed(1)

TODAY = datetime.date(2025, 12, 12)

def phone(i):
    return f"+41 79 {i:06d}"

BEHANDLUNG_CASES = [
    "Appendektomie Nachkontrolle",
    "Blutdruckeinstellung",
    "Diabetes Verlaufskontrolle",
    "Atemwegsinfekt Behandlung",
    "Herz-Kreislauf Check",
    "Wundversorgung",
    "Schmerztherapie",
    "Magen-Darm Untersuchung",
    "Präoperative Abklärung",
    "Postoperative Nachsorge"
]

STATION_NAMES = [
    "Chirurgie",
    "Innere Medizin",
    "Pädiatrie",
    "Notfall",
    "Intensivstation",
    "Radiologie"
]

RAUM_ARTEN = [
    "Operationssaal",
    "Untersuchungsraum"
]

MEDIKAMENTE = [
    (1, "Paracetamol", "Paracetamol"),
    (2, "Ibuprofen", "Ibuprofen"),
    (3, "Aspirin", "Acetylsalicylsäure"),
    (4, "Amoxicillin", "Amoxicillin"),
    (5, "Insulin", "Insulin")
]

# =========================
# Krankenhaus (3)
# =========================
for kh_id in range(1, 4):
    cur.execute("""
        INSERT INTO Krankenhaus VALUES (%s,%s,%s,%s,%s,%s)
    """, (
        kh_id,
        fake.street_name(),
        fake.building_number(),
        f"{1000 + kh_id}",
        fake.city(),
        f"Kantonsspital {kh_id}"
    ))
conn.commit()

# =========================
# Station (6)
# =========================
for sid, name in enumerate(STATION_NAMES, start=1):
    cur.execute("""
        INSERT INTO Station VALUES (%s,%s,%s,%s)
    """, (
        sid,
        name,
        random.randint(30, 60),
        ((sid - 1) % 3) + 1
    ))
conn.commit()

# =========================
# Raum (12)
# =========================
raum_id = 1
for sid in range(1, 7):
    for art in RAUM_ARTEN:
        cur.execute("""
            INSERT INTO Raum VALUES (%s,%s,%s)
        """, (raum_id, sid, art))
        raum_id += 1
conn.commit()

# =========================
# Patient (24)
# =========================
for pid in range(1, 25):
    cur.execute("""
        INSERT INTO Patient VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
    """, (
        pid,
        fake.first_name(),
        fake.last_name(),
        f"CH{pid:08d}",
        fake.date_of_birth(minimum_age=0, maximum_age=95),
        fake.street_name(),
        fake.building_number(),
        f"{1000 + pid}",
        fake.city(),
        f"EPD-{pid}",
        ((pid - 1) % 6) + 1
    ))
conn.commit()

# =========================
# Arzt (8)
# =========================
for aid in range(1, 9):
    cur.execute("""
        INSERT INTO Arzt VALUES (%s,%s,%s,%s,%s,%s)
    """, (
        aid,
        phone(aid),
        fake.first_name(),
        fake.last_name(),
        aid % 4 == 0,
        ((aid - 1) % 6) + 1
    ))
conn.commit()

# =========================
# Pflegekraft (12)
# =========================
for pk in range(1, 13):
    cur.execute("""
        INSERT INTO Pflegekraft VALUES (%s,%s,%s,%s,%s)
    """, (
        pk,
        phone(200 + pk),
        fake.first_name(),
        fake.last_name(),
        ((pk - 1) % 6) + 1
    ))
conn.commit()

# =========================
# Bett (18)
# =========================
bett_id = 1
for sid in range(1, 7):
    for i in range(3):
        cur.execute("""
            INSERT INTO Bett VALUES (%s,%s,%s,%s)
        """, (
            bett_id,
            "belegt" if i == 0 else "frei",
            bett_id if i == 0 else None,
            sid
        ))
        bett_id += 1
conn.commit()

# =========================
# Medikament (5)
# =========================
for mid, name, wirkstoff in MEDIKAMENTE:
    cur.execute("""
        INSERT INTO Medikament VALUES (%s,%s,%s)
    """, (mid, wirkstoff, name))
conn.commit()

# =========================
# Behandlung (24)
# =========================
for bid in range(1, 25):
    abgeschlossen = bid % 3 != 0
    if abgeschlossen:
        start = fake.date_between(datetime.date(2022, 1, 1), TODAY)
        end = start + datetime.timedelta(days=random.randint(1, 20))
        if end > TODAY:
            end = TODAY
    else:
        start = TODAY + datetime.timedelta(days=random.randint(1, 60))
        end = None

    cur.execute("""
        INSERT INTO Behandlung VALUES (%s,%s,%s,%s,%s,%s)
    """, (
        bid,
        abgeschlossen,
        BEHANDLUNG_CASES[(bid - 1) % len(BEHANDLUNG_CASES)],
        start,
        end,
        bid
    ))
conn.commit()

# =========================
# Event (24)
# =========================
for eid in range(1, 25):
    cur.execute("""
        INSERT INTO Event VALUES (%s,%s,%s,%s,%s)
    """, (
        eid,
        fake.time(),
        TODAY if eid % 3 == 0 else fake.date_between("-6m", "today"),
        ((eid - 1) % (raum_id - 1)) + 1,
        eid
    ))
conn.commit()

# =========================
# Arzt_Event (24)
# =========================
for eid in range(1, 25):
    cur.execute("""
        INSERT INTO Arzt_Event VALUES (%s,%s)
    """, (
        ((eid - 1) % 8) + 1,
        eid
    ))
conn.commit()

# =========================
# Event_Medikament (24)
# =========================
for eid in range(1, 25):
    cur.execute("""
        INSERT INTO Event_Medikament VALUES (%s,%s)
    """, (
        eid,
        ((eid - 1) % 5) + 1
    ))
conn.commit()


In [6]:
conn.rollback()
